## IMPORT 

In [1]:
import re
import nltk
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from unicode_tr import unicode_tr
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier, NeighborhoodComponentsAnalysis, LocalOutlierFactor

[nltk_data] Downloading package punkt to /home/emir/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/emir/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
from spacy.lang.tr import Turkish
from spacy.lang.tr.stop_words import STOP_WORDS
lemmatizer = Turkish().vocab.morphology.lemmatizer

In [3]:
def get_lemma(word):
    lemmas = lemmatizer(word, "noun")
    lemma = min(lemmas)
    return lemma.title() if word.istitle() else lemma

In [4]:
def remove_stopwords(word):
    nlp=Turkish()
    remove = ""
    lexeme = nlp.vocab[word]
    if lexeme.is_stop == True:
        return remove
    else:
        return word

In [5]:
remove_stopwords("ve")

''

## READ DATA


In [6]:
path = Path("/home/emir/MyProjects/DataI_Project/data/TTC-3600_Orj").iterdir()
# spor_data = Path("/home/emir/MyProjects/DataI_Project/data/TTC-3600_Orj/spor").iterdir()
# ekonomi_data = Path("/home/emir/MyProjects/DataI_Project/data/TTC-3600_Orj/ekonomi").iterdir()
# teknoloji_data = Path("/home/emir/MyProjects/DataI_Project/data/TTC-3600_Orj/teknoloji").iterdir()
# saglik_data = Path("/home/emir/MyProjects/DataI_Project/data/TTC-3600_Orj/saglik").iterdir()
# siyaset_data = Path("/home/emir/MyProjects/DataI_Project/data/TTC-3600_Orj/siyaset").iterdir()


In [7]:
print(list(Path("/home/emir/MyProjects/DataI_Project/data/TTC-3600_Orj").iterdir()))

[PosixPath('/home/emir/MyProjects/DataI_Project/data/TTC-3600_Orj/spor'), PosixPath('/home/emir/MyProjects/DataI_Project/data/TTC-3600_Orj/teknoloji'), PosixPath('/home/emir/MyProjects/DataI_Project/data/TTC-3600_Orj/ekonomi'), PosixPath('/home/emir/MyProjects/DataI_Project/data/TTC-3600_Orj/saglik'), PosixPath('/home/emir/MyProjects/DataI_Project/data/TTC-3600_Orj/kultursanat'), PosixPath('/home/emir/MyProjects/DataI_Project/data/TTC-3600_Orj/siyaset')]


In [8]:
data_list = []
data_df = pd.DataFrame()
for i in path:
    for j in i.iterdir():
        data = j.read_text()
        data_list.append((str(i).split("/")[-1],data))

In [9]:
data_df = pd.DataFrame(data_list, columns=["category","text"])

In [10]:
data_df

,category,text
0,spor,Mehmet Topal'ın eşinden örnek davranış\nFenerb...
1,spor,﻿\n\tSezonun tamamlanmasının ardından Aytaç Öd...
2,spor,﻿Geride bıraktığımız \n sezon \...
3,spor,"﻿TRABZON - İbrahim Hacıosmanoğlu, Trabzonspor ..."
4,spor,Kunter: 'Beşiktaş'ın hedefi şampiyonluk'\nBeşi...
...,...,...
3595,siyaset,11 milyon tablet geliyor\n11 milyon tabletin i...
3596,siyaset,﻿\n\tAynı anda kayıt yaptıran Bakan Kılıç ile ...
3597,siyaset,﻿\n\n \n \n \n \n \n\tHDP H...
3598,siyaset,AK Parti pankartına CHP tepkisi\nAdalet ve Kal...


In [11]:
data_df["category"].unique()

array(['spor', 'teknoloji', 'ekonomi', 'saglik', 'kultursanat', 'siyaset'],
      dtype=object)

In [12]:
mapping = {'spor':0, 
           'teknoloji':1, 
           'ekonomi':2, 
           'saglik':3, 
           'kultursanat':4, 
           'siyaset':5
          }

In [13]:
data_df.insert(1, "label",  None, True) 

In [14]:
data_df['label'] = data_df['category'].map(mapping)
data_df

,category,label,text
0,spor,0,Mehmet Topal'ın eşinden örnek davranış\nFenerb...
1,spor,0,﻿\n\tSezonun tamamlanmasının ardından Aytaç Öd...
2,spor,0,﻿Geride bıraktığımız \n sezon \...
3,spor,0,"﻿TRABZON - İbrahim Hacıosmanoğlu, Trabzonspor ..."
4,spor,0,Kunter: 'Beşiktaş'ın hedefi şampiyonluk'\nBeşi...
...,...,...,...
3595,siyaset,5,11 milyon tablet geliyor\n11 milyon tabletin i...
3596,siyaset,5,﻿\n\tAynı anda kayıt yaptıran Bakan Kılıç ile ...
3597,siyaset,5,﻿\n\n \n \n \n \n \n\tHDP H...
3598,siyaset,5,AK Parti pankartına CHP tepkisi\nAdalet ve Kal...


## PREPROCESSING

In [15]:
text_list = []
lemma = nltk.WordNetLemmatizer()
for text in data_df.text:
    text = unicode_tr(text).lower()
    text = re.sub(r"[^a-zğüı'şöç]", " ", text)
    text = nltk.word_tokenize(text)
#     text = [remove_stopwords(word) for word in text]
    text = [get_lemma(word) for word in text]
    text = " ".join(text)
    text_list.append(text)


In [34]:
max_features = 15000
count_vectorizer = TfidfVectorizer(max_features=max_features)
count_vectorizer.fit(text_list)
sparce_matrix = count_vectorizer.transform(text_list)

In [35]:
y= data_df.iloc[:,1].values
x= sparce_matrix.toarray()

## TRAIN TEST SPLIT

In [66]:
X_train,X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.20, random_state = 42)

## CREATE MODEL and TRAINING

## Naive Bayes

In [37]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train, Y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [38]:
print("accurancy: ", nb.score(X_test,Y_test))

accurancy:  0.9277777777777778


## Logistic Regression


In [39]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, Y_train)

# get accuracy score
pred_lr = lr.predict(X_test)
acc_lr = accuracy_score(pred_lr, Y_test)
print(acc_lr)

/home/emir/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/emir/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.9277777777777778


## KNN

In [75]:
knn = KNeighborsClassifier()
knn.fit(X_train, Y_train)
y_pred_test = knn.predict(X_test)
y_pred_train = knn.predict(X_train)
    
cm_test = confusion_matrix(Y_test, y_pred_test)
cm_train = confusion_matrix(Y_train, y_pred_train)

In [77]:
 acc_test = accuracy_score(Y_test, y_pred_test)
acc_train = accuracy_score(Y_train, y_pred_train)
print("Test Score: {}, Train Score: {}".format(acc_test, acc_train))
print()
print("CM Test", cm_test)
print("CM Train", cm_train)

Test Score: 0.8791666666666667, Train Score: 0.9277777777777778

CM Test [[141   0   1   1   3   1]
 [  1  99   4   9   5   1]
 [  3   4  94   3   7   8]
 [  0   0   2 103   4   2]
 [  4   4   2   1 102   4]
 [  2   1   8   2   0  94]]
CM Train [[430   4   6   3   4   6]
 [  8 435  13  11   9   5]
 [  8  10 436   5   4  18]
 [  2   3  12 461   7   4]
 [  8   8  14   3 442   8]
 [  4   1   8   8   4 468]]


## LSTM

In [19]:
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout
from sklearn.metrics import mean_squared_error
from keras.utils import to_categorical
from sklearn import metrics

Using TensorFlow backend.
/home/emir/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/emir/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/emir/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/emir/.local/lib/python3.6/site-packages/tensorflow/python/framewo

In [41]:
Y_train = to_categorical(Y_train)

In [42]:
Y_train

array([[0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       ...,
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.]], dtype=float32)

In [43]:
Y_train.shape

(2880, 6)

In [44]:
X_train = np.reshape(X_train, (X_train.shape[0],1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0],1, X_test.shape[1]))

In [45]:
X_train.shape

(2880, 1, 15000)

In [182]:
# import numpy as np
# X_train = np.array(X_train).reshape(-1,1)
# Y_train = np.array(Y_train).reshape(-1)

In [24]:
X_train.shape

(2880, 1, 15000)

In [49]:
Y_test = to_categorical(Y_test)

In [26]:
Y_test


array([[0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       ...,
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0.]], dtype=float32)

In [27]:
X_train.shape

(2880, 1, 15000)

In [28]:
X_test.shape

(720, 1, 15000)

In [29]:
Y_test.shape

(720, 6)

In [190]:
Y_train.shape

(17280,)

In [191]:
print(Y_train)

[0. 0. 0. ... 0. 0. 1.]


In [46]:
model = Sequential()
model.add(LSTM(30, input_shape=(1,15000)))
model.add(Dropout(0.5))
model.add(Dense(6, activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [47]:
hisory=model.fit(X_train, Y_train, epochs=10, batch_size=32)

Epoch 1/10
2880/2880 [==============================] - 4s 1ms/step - loss: 1.7271 - accuracy: 0.7229
Epoch 2/10
2880/2880 [==============================] - 4s 1ms/step - loss: 1.3555 - accuracy: 0.9142
Epoch 3/10
2880/2880 [==============================] - 4s 1ms/step - loss: 0.7785 - accuracy: 0.9618
Epoch 4/10
2880/2880 [==============================] - 4s 1ms/step - loss: 0.4150 - accuracy: 0.9757
Epoch 5/10
2880/2880 [==============================] - 3s 1ms/step - loss: 0.2548 - accuracy: 0.9837
Epoch 6/10
2880/2880 [==============================] - 3s 1ms/step - loss: 0.1667 - accuracy: 0.9910
Epoch 7/10
2880/2880 [==============================] - 4s 1ms/step - loss: 0.1210 - accuracy: 0.9948
Epoch 8/10
2880/2880 [==============================] - 4s 1ms/step - loss: 0.0913 - accuracy: 0.9958
Epoch 9/10
2880/2880 [==============================] - 4s 1ms/step - loss: 0.0673 - accuracy: 0.9972
Epoch 10/10
2880/2880 [==============================] - 4s 1ms/step - loss: 0.054

In [50]:
model.evaluate(X_test, Y_test)

720/720 [==============================] - 0s 424us/step


[0.2145328487786982, 0.9347222447395325]

## test new sentences


In [62]:
text1 = ["Antalya'nın Muratpaşa ilçesinde yaşayan emekli Ahmet Turan Gürel de sunulan kredi avantajından haberdar olduktan sonra gerekli başvuruyu yaptığını dile getirdi.Reklamda 3 bin ile 10 bin lira arasında ihtiyaç kredisi verildiğini görünce mutlu olduğunu aktaran Gürel, şöyle konuştu",
        'Dünyaca ünlü piyanist Fazıl Say, 23 Nisanda Türkiye Büyük Millet Meclisinin (TBMM) kuruluşunun 100. yılı dolayısıyla yaptığı yeni besteyi sosyal medya hesaplarında paylaştı Say, Şükran Türküsü adını verdiği bestesini "23 Nisan Ulusal Egemenlik ve Çocuk Bayramı kutlu olsun" notuyla paylaştı.',
        'Menajer Ömer Uzun, Mohamed Elnenyin Beşiktaş ta kalıp kalmayacağıyla ilgili, "Gerek Arsenal tarafında gerek Beşiktaş tarafında salgın sonrasında birtakım şeyler beklemede. Şu ana kadar resmi bir görüşme yapamadık ancak şunu biliyorum ki gerek teknik heyet gerek Beşiktaş, Elnenyi bir sezon daha takımda tutmak istiyor. İşler biraz daha netleştikten sonra da Arsenal ile de oyuncunun gelecek sezon planlamasıyla ilgili birtakım görüşmeler yapılacaktır" dedi',
        "Yeni tip corona virüs salgını nedeniyle yaşanan ölümlerin sayısı 185 bini aşarken, bilim insanları ve ilaç firmaları küresel salgını durdurmak için birbiriyle yarışıyor. Dünya Sağlık Örgütü (DSÖ), dünya genelinde 83 yeni tip corona virüs (Covid-19), aşısının geliştirildiğini ve bunların altısının insan denemeleri aşamasında olduğunu belirtti."]

In [63]:
text1 = [unicode_tr(i).lower() for i in text1]

In [64]:
text1

["antalya'nın muratpaşa ilçesinde yaşayan emekli ahmet turan gürel de sunulan kredi avantajından haberdar olduktan sonra gerekli başvuruyu yaptığını dile getirdi.reklamda 3 bin ile 10 bin lira arasında ihtiyaç kredisi verildiğini görünce mutlu olduğunu aktaran gürel, şöyle konuştu",
 'dünyaca ünlü piyanist fazıl say, 23 nisanda türkiye büyük millet meclisinin (tbmm) kuruluşunun 100. yılı dolayısıyla yaptığı yeni besteyi sosyal medya hesaplarında paylaştı say, şükran türküsü adını verdiği bestesini "23 nisan ulusal egemenlik ve çocuk bayramı kutlu olsun" notuyla paylaştı.',
 'menajer ömer uzun, mohamed elnenyin beşiktaş ta kalıp kalmayacağıyla ilgili, "gerek arsenal tarafında gerek beşiktaş tarafında salgın sonrasında birtakım şeyler beklemede. şu ana kadar resmi bir görüşme yapamadık ancak şunu biliyorum ki gerek teknik heyet gerek beşiktaş, elnenyi bir sezon daha takımda tutmak istiyor. işler biraz daha netleştikten sonra da arsenal ile de oyuncunun gelecek sezon planlamasıyla ilgili 

In [65]:
text1 = [re.sub(r"[^a-zğüı'şöç]", " ", i) for i in text1]
text1

["antalya'nın muratpaşa ilçesinde yaşayan emekli ahmet turan gürel de sunulan kredi avantajından haberdar olduktan sonra gerekli başvuruyu yaptığını dile getirdi reklamda   bin ile    bin lira arasında ihtiyaç kredisi verildiğini görünce mutlu olduğunu aktaran gürel  şöyle konuştu",
 'dünyaca ünlü piyanist fazıl say     nisanda türkiye büyük millet meclisinin  tbmm  kuruluşunun      yılı dolayısıyla yaptığı yeni besteyi sosyal medya hesaplarında paylaştı say  şükran türküsü adını verdiği bestesini     nisan ulusal egemenlik ve çocuk bayramı kutlu olsun  notuyla paylaştı ',
 'menajer ömer uzun  mohamed elnenyin beşiktaş ta kalıp kalmayacağıyla ilgili   gerek arsenal tarafında gerek beşiktaş tarafında salgın sonrasında birtakım şeyler beklemede  şu ana kadar resmi bir görüşme yapamadık ancak şunu biliyorum ki gerek teknik heyet gerek beşiktaş  elnenyi bir sezon daha takımda tutmak istiyor  işler biraz daha netleştikten sonra da arsenal ile de oyuncunun gelecek sezon planlamasıyla ilgili 

In [66]:
text1=count_vectorizer.transform(text1)
text1=text1.toarray()

In [67]:
text1.shape

(4, 15000)

In [68]:
X_test.shape

(720, 15000)

In [69]:
predictions = nb.predict(text1)

In [70]:
mapping_list = list(mapping.keys())

In [72]:
for j in predictions:
    pred = mapping_list[j]
    print(pred)

ekonomi
kultursanat
spor
saglik
